In [142]:
import numpy as np
import scipy.signal

In [143]:
# The azimuth angles (in degrees) of the loudspeakers.
# Angles must be in counter-clockwise order.
emitter_angles = np.array([30, 110, 150, -150, -110, -30])
emitter_channel_ordering = np.array([0, 4, 6, 7, 5, 1])

In [144]:
# Unit vectors for the loudspeakers. Not that the matrix is constructed as the transpose of the vectors.
# When P' = G * L, the result of P' is the correct p = g1 L1 + g2 L2 by scaled vector addition.
# The projections of the unit vectors sum to the unit vector in the direction of the source.
L = np.array((np.cos(np.radians(emitter_angles)), np.sin(np.radians(emitter_angles)))).T

In [145]:
# Create successive pairs of speaker angles until looping around. The pair (n1, n2) where 
# n1 is 'more clock-wise' than n2.
panning_pairs = np.vstack([(i, i + 1) for i in range(len(emitter_angles) - 1)] + [(len(emitter_angles) - 1, 0)])

In [146]:
# The azimuth angles (in degrees) of the sources.
source_angles = np.array([0, 70, 130, 180, -130, -70])

In [147]:
# Calculate the cartesian directional unit vectors for the sound sources.
P = np.array((np.cos(np.radians(source_angles)), np.sin(np.radians(source_angles))))

In [7]:
# G is the matrix that contains the calculated gains between each emitter within
# a panning pair of emitters, for each emitter.
#
# For M sound sources, for N panning pairs, [n1, n2] where n1 is 'more clock-wise' than n2.
G = np.zeros((len(source_angles), len(emitter_angles), 2))

In [8]:
# For each source, calculate the gains matrix for all panning pairs.

# The directional unit vectors for a specific panning pair.
Ln1n2 = np.zeros((2, 2))

for m in range(len(source_angles)):
    pT = P[:, m].reshape(1, 2)
    for n in range(len(emitter_angles)):
        panning_pair = list(panning_pairs[n])
        Ln1n2[:,:] = L[panning_pair,:]
        # del panning_pair
        G[m, n, :] = pT @ np.linalg.inv(Ln1n2)

        # Normalize the gains
        G[m, n, :] /= np.linalg.norm(G[m, n, :])
    del pT

# del Ln1n2
# del m, n

In [9]:
class PannedSource:
    def __init__(self, source_angle, group, gains):
        self.source_angle = source_angle
        self.group = group
        self.gains = gains
        self.gains[gains < 0.000001] = 0.0
        

    def __str__(self):
        return f'{self.source_angle} = {self.gains[0]} * {self.group[0]} + {self.gains[1]} * {self.group[1]}'
    
    def pan(self, source, input):
        '''
        source.shape == (m, 1)
        input.shape == (m, n) where m is the number of samples and n is the number of channels/speakers.

        Returns an m x n array with panned source correctly mixed into the appropriate channels of input.
        Specifically, 
            output[:,self.group] = self.gains * source + input[self.group]
        '''

        return self.gains * np.repeat(source, 2).reshape(len(source), 2) + input[:,self.group]

valid_pannings = np.logical_and(G > 0, G < 1.00001)
panned_sources = [PannedSource(source_angle=source_angles[m],
                                group=panning_pairs[n],
                                gains=G[m, n, :]) 
                                for m in range(len(source_angles)) 
                                for n in range(len(emitter_angles)) 
                                if valid_pannings[m,n,:].all()]
# del valid_pannings
# del panning_pairs

In [ ]:
[str(panned_source) for panned_source in panned_sources]

In [11]:
fs = 48000
T = 10
t = np.linspace(0, T, fs * T)

frequencies = [261.63, 329.63, 392.00, 523.25, 659.25, 783.99]

sources = [np.sin(2 * np.pi * f * t) for f in frequencies[:len(source_angles)]]

In [23]:
panned_output = np.zeros((fs * T, len(emitter_angles)))

for i in range(len(sources)):
    panned_source = panned_sources[i]
    panned_output[:,panned_source.group] += 10.0**(-3.0/20.0) * panned_source.pan(sources[i], panned_output)

output = np.zeros((T * fs, 8))

for i in range(len(emitter_angles)):
    divisor = np.max(np.abs(panned_output[:,i]))
    if math.isclose(divisor, 0.0):
        output[:,emitter_channel_ordering[i]] = panned_output[:,i]
    else:
        output[:,emitter_channel_ordering[i]] = panned_output[:,i] / divisor
        
# Write output as a 32-bit float WAV file.
scipy.io.wavfile.write('output.wav', fs, output.astype(np.float32))
